In [49]:
# This script converts the trained "model.h5" into OpenVINO MO.
# https://docs.openvino.ai/2022.3/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html#keras-h5 
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from IPython.display import Markdown
from openvino.runtime import Core

In [50]:
model_h5_dir = Path("./files/A_20240630")
model_h5_path = model_h5_dir / "model.h5"

In [51]:
from metrics import iou, dice_coef, dice_loss, bce_dice_loss
from tensorflow.keras.utils import CustomObjectScope

# Load the model
with CustomObjectScope({
        'iou':iou,
        'dice_coef':dice_coef,
        'dice_loss':dice_loss,
        'bce_dice_loss': bce_dice_loss
    }):
    try:
        h5_model = tf.keras.models.load_model(model_h5_path)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        

Model loaded successfully.


In [52]:
saved_model_path =  model_h5_dir /'model'
tf.saved_model.save(h5_model,saved_model_path)


2024-07-10 20:19:40.220751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-10 20:19:40.756781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-10 20:19:40.908610: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2024-07-10

INFO:tensorflow:Assets written to: files/A_20240630/model/assets


INFO:tensorflow:Assets written to: files/A_20240630/model/assets


In [53]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path)

In [54]:
fn = str(saved_model_path /'nanonet-fp32-shape-1-256-256-3-model.xml')
ov.save_model(ov_model, fn)

In [55]:
# Open the file and read the first 20 lines
with open(fn, 'r') as file:
    for i in range(50):
        line = file.readline()
        print(line.strip())  # Using strip() to remove any extra newline characters

<?xml version="1.0"?>
<net name="TensorFlow_Frontend_IR" version="11">
<layers>
<layer id="0" name="input_image" type="Parameter" version="opset1">
<data shape="?,256,256,3" element_type="f32" />
<output>
<port id="0" precision="FP32" names="input_image">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
</output>
</layer>
<layer id="1" name="Constant_117359" type="Const" version="opset1">
<data element_type="i64" shape="4" offset="0" size="32" />
<output>
<port id="0" precision="I64">
<dim>4</dim>
</port>
</output>
</layer>
<layer id="2" name="Transpose_117360" type="Transpose" version="opset1">
<input>
<port id="0" precision="FP32">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
<port id="1" precision="I64">
<dim>4</dim>
</port>
</input>
<output>
<port id="2" precision="FP32">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
</port>
</output>
</layer>
<layer id="3" name="Multiply_134554_compressed" type="Const" version="opset1">
<data element_

In [56]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path)

In [57]:
# Convert Input Layout to N,C,H,W
prep = ov.preprocess.PrePostProcessor(ov_model)
input_name = ov_model.input().get_any_name()
print("input_name is", input_name)
prep.input(input_name).tensor().set_layout(ov.Layout("nchw"))
prep.input(input_name).model().set_layout(ov.Layout("nhwc"))
ov_model = prep.build()


input_name is input_image


In [58]:
fn = str(saved_model_path /'nanonet-fp32-shape-1-3-256-256-model.xml')
ov.save_model(ov_model, fn)

In [59]:
# Open the file and read the first 20 lines
with open(fn, 'r') as file:
    for i in range(50):
        line = file.readline()
        print(line.strip())  # Using strip() to remove any extra newline characters

<?xml version="1.0"?>
<net name="TensorFlow_Frontend_IR" version="11">
<layers>
<layer id="0" name="input_image" type="Parameter" version="opset1">
<data shape="?,3,256,256" element_type="f32" />
<output>
<port id="0" precision="FP32" names="input_image">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
<rt_info>
<attribute name="layout" version="0" layout="[N,C,H,W]" />
</rt_info>
</port>
</output>
</layer>
<layer id="1" name="Constant_153914" type="Const" version="opset1">
<data element_type="i64" shape="4" offset="0" size="32" />
<output>
<port id="0" precision="I64">
<dim>4</dim>
</port>
</output>
</layer>
<layer id="2" name="Transpose_153915" type="Transpose" version="opset1">
<input>
<port id="0" precision="FP32">
<dim>-1</dim>
<dim>3</dim>
<dim>256</dim>
<dim>256</dim>
</port>
<port id="1" precision="I64">
<dim>4</dim>
</port>
</input>
<output>
<port id="2" precision="FP32">
<dim>-1</dim>
<dim>256</dim>
<dim>256</dim>
<dim>3</dim>
</port>
</output>
</layer>
<layer id="3"